In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    "../data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="training"
)
val_generator = train_datagen.flow_from_directory(
    "../data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


In [3]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(150, 150, 3),
    include_top=False,
    weights="imagenet"
)

# Freeze base model
base_model.trainable = False

# Add custom classifier
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(train_generator, validation_data=val_generator, epochs=5)

c:\Users\Mana\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 103s 497ms/step - accuracy: 0.5334 - loss: 0.7012 - val_accuracy: 0.5091 - val_loss: 0.6785
Epoch 2/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 55s 272ms/step - accuracy: 0.5664 - loss: 0.6782 - val_accuracy: 0.6265 - val_loss: 0.6640
Epoch 3/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 55s 273ms/step - accuracy: 0.5823 - loss: 0.6711 - val_accuracy: 0.6346 - val_loss: 0.6576
Epoch 4/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 56s 276ms/step - accuracy: 0.5759 - loss: 0.6739 - val_accuracy: 0.6052 - val_loss: 0.6617
Epoch 5/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 55s 276ms/step - accuracy: 0.5954 - loss: 0.6670 - val_accuracy: 0.6390 - val_loss: 0.6515


In [8]:
acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f"Train Accuracy: {acc:.2f}, Validation Accuracy: {val_acc:.2f}")

model.save("../models/cats_dogs_resnet.h5")

Train Accuracy: 0.60, Validation Accuracy: 0.64
